# LeetCode Rating graph

## Import libraries

In [1]:
import requests
import time
import json
import math
import os
import threading
import datetime

import pandas as pd

from bokeh.plotting import figure
from bokeh.io import output_notebook, push_notebook, show
from bokeh.models import DatetimeTickFormatter, BoxAnnotation, Legend
from bokeh.embed import file_html
from bokeh.plotting import output_file, save
from bokeh.layouts import column

In [2]:
print('Pandas version: {0}'.format(pd.__version__))

Pandas version: 0.25.1


## Headers

In [3]:
headers = {
    'authority': 'leetcode.com',
    'accept': 'application/json, text/javascript, */*; q=0.01',
    'x-newrelic-id': 'UAQDVFVRGwEAXVlbBAg=',
    'x-requested-with': 'XMLHttpRequest',
    'user-agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.85 Safari/537.36',
    'content-type': 'application/json',
    'sec-gpc': '1',
    'sec-fetch-site': 'same-origin',
    'sec-fetch-mode': 'cors',
    'sec-fetch-dest': 'empty',
    'accept-language': 'en-US,en;q=0.9',
    'cookie': 'csrftoken=SYt4auAHmprCYhMeGDZwbnE0hWoSLKRLHb0A7mMpbvoAEPvjYcIdL5LtaWhQJv6B; __cfduid=dc8614cf2c524abada1e54cfa8e118d271619065816; __cf_bm=08dec7596eec629845d4f3f39086483d491e2c40-1619324757-1800-AYKgjE4h5oPn/k7bslI317JZOCKGqrKvRQEJSKFM8EyY57yFeDgrM0KTqzs/c5RUfzN8A7VceydacsvxJjrmrp4=',
}

## Queries

```json
"operationName": "getContestRankingData"
"query": "query getContestRankingData($username: String!) {\n  userContestRanking(username: $username) {\n    attendedContestsCount\n    rating\n    globalRanking\n    __typename\n  }\n  userContestRankingHistory(username: $username) {\n    contest {\n      title\n      startTime\n      __typename\n    }\n    rating\n    ranking\n    __typename\n  }\n}\n"
"variables": "{ username: \"snehith26" }"
```

## Retreive ratings

In [28]:
class UserRankingAndRating:
    def __init__(self, title: str, rating: float, ranking: int, timestamp: int, date: str):
        title = title
        rating = rating
        ranking = ranking
        timestamp = timestamp
        date = date

def getContestSlug(contest_title: str):
    return contest_title.lower().replace(' ', '-')

class GetContestInfo:
    def __init__(self):
        self.filename = 'contest_info.csv'
        self.updated = False
        # Read CSV if it exists
        if (os.path.exists(self.filename)):
            self.dataframe = pd.read_csv(self.filename)
        else:
            self.dataframe = pd.DataFrame(columns=['Contest title', 'Contest slug', 'Number of participants'])
    
    def numberOfParticipantsInContest(self, contest_title: str, is_title: bool = True):
        slug = getContestSlug(contest_title)
        params = (
            ('pagination', '1'),
            ('region', 'global'),
        )
        num_participants = 1
        if (self.dataframe.loc[self.dataframe['Contest slug'] == slug].shape[0] == 0):
            # https://leetcode.com/contest/api/ranking/weekly-contest-246/?pagination=1&region=global
            data = json.loads(requests.get('https://leetcode.com/contest/api/ranking/{}/'.format(slug), headers=headers, params=params).content)
            new_data = [contest_title, slug, 0]
            if (str(data) != '{}'):
                new_data[2] = int(data['user_num'])
            else:
                new_data[2] = 1
            num_participants = new_data[2]
            self.dataframe.loc[len(self.dataframe)] = new_data
            self.updated = True
        elif (self.dataframe.loc[self.dataframe['Contest slug'] == slug].shape[0] == 1):
            pass
        else:
            pass
        return num_participants
    
    def __del__(self):
        if self.updated == True:
            self.dataframe.to_csv(self.filename, index=False)

            
def getUserLatestPosts(username: str, limit: int = 10):
    # getRecentPosts
    query = '{"operationName":"getRecentPosts","variables":{"username":\"'+username+'\","limit":\"'+str(limit)+'\"},"query":"query getRecentPosts($username: String!, $limit: Int) {\\n userRecentTopics(username: $username, limit: $limit) {\\n    id\\n     title\\n    post {\\n     creationDate\\n     }\\n }\\n }\\n"}'
    response = requests.post('https://leetcode.com/graphql', headers=headers, data=query).content
    response = json.loads(response)
    print(response)
    infos = {
        'Published': [],
        'Title': [],
        'Url': [],
        'Last modified': [],
    }
    url_prefix = 'https://leetcode.com/discuss/topic/'
    for item in response['data']['userRecentTopics']:
        infos['Published'].append(item['post']['creationDate'])
        infos['Last modified'].append(item['post']['creationDate'])
        infos['Title'].append(item['title'])
        infos['Url'].append(url_prefix + str(item['id']))
    print(infos['Url'])


def getUserRankingAndRating(username: str):
    query = '{"operationName":"getContestRankingData","variables":{"username":\"'+username+'\"},"query":"query getContestRankingData($username: String!) {\\n  userContestRankingHistory(username: $username) {\\n    contest {\\n      title\\n      startTime\\n      __typename\\n    }\\n    rating\\n    ranking\\n    __typename\\n  }\\n}\\n"}'
    response = requests.post('https://leetcode.com/graphql', headers=headers, data=query).content
    response = json.loads(response)
    infos = {
        'Date': [],
        'Contest title': [],
        'Rating': [],
        'Ranking': [],
        'Total number of participants': [],
        'Timestamp': []
    }
    get_contest_info = GetContestInfo()
    for item in response['data']['userContestRankingHistory']:
        timestamp = datetime.datetime.fromtimestamp(item['contest']['startTime']).strftime('%Y-%m-%d %H:%M:%S')
#         info = UserRankingAndRating(
#             title=item['contest']['title'],
#             rating=item['rating'],
#             ranking=item['ranking'],
#             timestamp=item['contest']['startTime'],
#             date=timestamp
#         )
        infos['Contest title'].append(item['contest']['title'])
        infos['Rating'].append(item['rating'])
        infos['Ranking'].append(item['ranking'])
        infos['Timestamp'].append(item['contest']['startTime'])
        infos['Date'].append(timestamp)
        # infos['Total number of participants'].append(1)
        infos['Total number of participants'].append(get_contest_info.numberOfParticipantsInContest(item['contest']['title']))
        # infos['Total number of participants'].append(numberOfParticipantsInContest(getContestSlug(item['contest']['title'])))
    df = pd.DataFrame(infos, columns = ['Date', 'Contest title', 'Rating', 'Ranking', 'Total number of participants', 'Timestamp'])
    return df

## Get user's latest posts

In [29]:
getUserLatestPosts('snehith26')

{'data': {'userRecentTopics': [{'id': 1010187, 'title': '[Python] Place largest k-th jobs into k-buckets - 80ms', 'post': {'creationDate': 1610264036}}, {'id': 419609, 'title': 'Simple Sliding Window Approach with left and right pointers', 'post': {'creationDate': 1572764229}}]}}
['https://leetcode.com/discuss/topic/1010187', 'https://leetcode.com/discuss/topic/419609']


## Get ratings for all required users

In [5]:
usernames = ['snehith26', 'sonapraneeth-a', 'jayaprakash_a']
infos = []
for username in usernames:
    infos.append(getUserRankingAndRating(username))

## Plot ratings

In [6]:
output_notebook()
rating_plot = figure(plot_width=1200, plot_height=600, x_axis_type="datetime")
legend = Legend(items=[])
rating_plot.add_layout(legend, 'right')
colors = ['red', 'green', 'blue']
for idx in range(len(infos)):
    info = infos[idx]
    rating_plot.line(
        # Format: 2016-08-28 07:00:00
        info['Date'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S')), 
        info['Rating'], 
        line_width=1, 
        color=colors[idx], 
        legend_label='{0}'.format(usernames[idx])
    )
    rating_plot.xaxis.formatter=DatetimeTickFormatter(
        hours=["%m/%d/%Y"]
    )
    rating_plot.title.text = "Plot for user rating"
handle = show(rating_plot, notebook_handle=True)
push_notebook(handle=handle)

Loading BokehJS ...

## References

- [LCPredict](https://github.com/s-i-d-d-i-s/LC-Predict/blob/main/Notebooks/Leetcode%20Rating%20Predictor.ipynb)
- [strptime](https://www.programiz.com/python-programming/datetime/strptime)